<a href="https://colab.research.google.com/github/kiritoamk/Machine-Learning/blob/main/Wenchuan_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import os
from PIL import Image

In [29]:
def load_images_from_uploaded(img_dir, mask_dir, img_size=(128, 128)):
    images = []
    masks = []
    for img_file in os.listdir(img_dir):
        img_path = os.path.join(img_dir, img_file)
    for mask_file in os.listdir(mask_dir):
        mask_path = os.path.join(mask_dir, mask_file)  # Assumes masks have the same name as images

         # Check if the corresponding mask file exists
        if not os.path.exists(mask_path):
            print(f"Mask for {img_file} not found. Skipping this file.")
            continue

        img = Image.open(img_path)
        mask = Image.open(mask_path)

        # Resize to match model input size
        img = img.resize(img_size)
        mask = mask.resize(img_size)

        # Normalize image and mask values
        images.append(np.array(img) / 255.0)  # Scale images to [0, 1]
        masks.append(np.array(mask) / 255.0)  # Scale masks to [0, 1]

    return np.array(images), np.array(masks)

In [35]:
img_dir = '/content/img'        # Folder where you upload images
mask_dir = '/content/mask'     # Folder where you upload masks

In [38]:
images, masks = load_images_from_uploaded(img_dir, mask_dir)

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(images, masks, test_size=0.2, random_state=42)

In [57]:
import tensorflow as tf
from tensorflow.keras import layers

def unet_model(input_size=(128, 128, 3)):
    inputs = tf.keras.Input(input_size)

    # Encoder
    c1 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(p1)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(p2)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    # Bottleneck
    c4 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p3)

    # Decoder with Conv2DTranspose
    u1 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c4)
    u1 = layers.Concatenate()([u1, c3])
    u1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u1)

    u2 = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(u1)
    u2 = layers.Concatenate()([u2, c2])
    u2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u2)

    u3 = layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(u2)
    u3 = layers.Concatenate()([u3, c1])
    u3 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(u3)

    # Output layer for binary segmentation
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(u3)

    model = tf.keras.Model(inputs, outputs)
    return model

model = unet_model()


In [58]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [49]:
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=8,
    validation_data=(X_val, y_val)
)

Epoch 1/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 21s 881ms/step - accuracy: 0.7893 - loss: 0.5254 - val_accuracy: 0.9590 - val_loss: 0.0026
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 19s 845ms/step - accuracy: 0.9531 - loss: 0.0030 - val_accuracy: 0.9590 - val_loss: 0.0026
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 19s 747ms/step - accuracy: 0.9528 - loss: 0.0030 - val_accuracy: 0.9590 - val_loss: 0.0026
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 13s 720ms/step - accuracy: 0.9436 - loss: 0.0036 - val_accuracy: 0.9590 - val_loss: 0.0026
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 25s 960ms/step - accuracy: 0.9469 - loss: 0.0034 - val_accuracy: 0.9590 - val_loss: 0.0026
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 15s 814ms/step - accuracy: 0.9455 - loss: 0.0034 - val_accuracy: 0.9590 - val_loss: 0.0026
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 20s 802ms/step - accuracy: 0.9453 - loss: 0.0035 - val_accuracy: 0.9590 - val_loss: 0.0026
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 20s 759ms/step - accuracy: 0.9489 - loss: 0.0032 - val_accu

In [50]:
model.save('landslide_segmentation_model.h5')

In [51]:
import matplotlib.pyplot as plt